In [ ]:
import pandas as pd
import numpy as np
import quilt3
from pathlib import Path
from aicsimageio.aics_image import AICSImage
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter

In [ ]:
from tqdm import tqdm

In [ ]:
pkg = quilt3.Package.browse("aics/hipsc_single_cell_image_dataset", registry="s3://allencell")
meta_df = pkg["metadata.csv"]()

In [ ]:
meta_df.set_index("CellId", inplace=True)

In [ ]:
meta_df.to_csv("/allen/aics/animated-cell/Saurabh/cellpack_PILR_analysis/data/variance_dataset.csv")

In [ ]:
print(meta_df.columns)

In [ ]:
pex_data = meta_df[meta_df["structure_name"] == 'SLC25A17']

In [ ]:
meta_df

get cellIDs in 8d sphere

In [ ]:
df_path = "/allen/aics/animated-cell/Saurabh/cellpack-analysis/data/8dsphere_ids.csv"

In [ ]:
df_cellID = pd.read_csv(df_path)

In [ ]:
df_cellID

In [ ]:
df_cellID.set_index("structure", inplace=True)

In [ ]:
str_cellid = df_cellID.loc["SLC25A17", "CellIds"].split(",")

In [ ]:
cellid_list = []
for cellid in str_cellid:
    cellid_list.append(int(cellid.replace("[", "").replace("]", "")))


In [ ]:
print(*cellid_list)

In [ ]:
num_sample = 15
data = pex_data.sample(num_sample)
data = data.reset_index(drop=True)

In [ ]:
data = pex_data[pex_data["CellId"].isin(cellid_list)].reset_index(drop=True)
data.shape

In [ ]:
data = pex_data.reset_index(drop=True)

In [ ]:
data.structure_name.unique()

In [ ]:
data.shape

In [ ]:
# prepare file paths
save_path = Path("/allen/aics/animated-cell/Saurabh/cellpack/data/images_for_PILR_8d")
save_path.mkdir(exist_ok=True)
raw_path = save_path / Path("seg_image")
raw_path.mkdir(exist_ok=True)
mesh_path = save_path / Path("mesh_image")
mesh_path.mkdir(exist_ok=True)

In [ ]:
for row in tqdm(data.itertuples()):
    subdir_name = row.crop_seg.split("/")[0]
    file_name = row.crop_seg.split("/")[1]
    local_fn = raw_path / f"{row.structure_name}_{row.CellId}_ch_{row.ChannelNumberStruct}_crop_seg_original.tiff"
    # pkg[subdir_name][file_name].fetch(local_fn)